In [1]:
import datasets
from datasets import load_from_disk
from datasets import concatenate_datasets

/home/haznitrama/scale-resources/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Dataset Work

In [20]:
cbn_wiki_id = load_from_disk("dataset/paralel_id_cbn_16k")
cbn_wiki_jv = load_from_disk("dataset/paralel_jv_cbn_3k")
cbn_wiki_su = load_from_disk("dataset/paralel_su_cbn_3k")

In [21]:
id_wiki_id = load_from_disk("synthetic/id_titles/id_wiki-id/translated")
id_wiki_jv = load_from_disk("synthetic/id_titles/id_wiki-jv/translated")
id_wiki_su = load_from_disk("synthetic/id_titles/id_wiki-su/translated")

In [22]:
from datasets import concatenate_datasets

id_cbn = concatenate_datasets([cbn_wiki_id, id_wiki_id])
jv_cbn = concatenate_datasets([cbn_wiki_jv, id_wiki_jv])
su_cbn = concatenate_datasets([cbn_wiki_su, id_wiki_su])

In [25]:
len(su_cbn)

28551

In [26]:
id_cbn.save_to_disk("dataset/paralel_id_cbn_127k")
jv_cbn.save_to_disk("dataset/paralel_jv_cbn_24k")
su_cbn.save_to_disk("dataset/paralel_su_cbn_28k")

Saving the dataset (1/1 shards): 100%|██████████| 28551/28551 [00:00<00:00, 198725.47 examples/s]


In [27]:
paralel_bali_dict = load_from_disk("dataset/paralel_dataset_from_bali_dict")  

In [29]:
paralel_bali_dict = paralel_bali_dict.remove_columns(["prompt"])

In [31]:
paralel_60k = load_from_disk("dataset/paralel_dataset_60k")

In [33]:
def convert_dataset(data):
    data["id"] = data["custom_id"]
    return data

In [35]:
paralel_60k = paralel_60k.map(convert_dataset, remove_columns=["custom_id", "text", "prompt_text"])

Map: 100%|██████████| 59257/59257 [00:08<00:00, 7163.49 examples/s] 


In [39]:
paralel_300k = load_from_disk("dataset/paralel_dataset_300k")

In [40]:
paralel_300k

Dataset({
    features: ['id', 'cirebonese', 'indonesian', 'balinese'],
    num_rows: 296203
})

In [41]:
paralel_bali_dict

Dataset({
    features: ['id', 'balinese', 'indonesian', 'cirebonese'],
    num_rows: 22248
})

In [42]:
paralel_60k

Dataset({
    features: ['indonesian', 'cirebonese', 'balinese', 'id'],
    num_rows: 59257
})

In [100]:
348695 + 43818

392513

In [95]:
paralel_377k = load_from_disk("dataset/paralel_3_lang/paralel_dataset_377k_dedup")

In [96]:
paralel_44k = load_from_disk("dataset/paralel_3_lang/paralel_dataset_44k")

In [97]:
paralel_377k

Dataset({
    features: ['id', 'cirebonese', 'indonesian', 'balinese'],
    num_rows: 348695
})

In [99]:
paralel_44k

Dataset({
    features: ['id', 'indonesian', 'cirebonese', 'balinese'],
    num_rows: 43818
})

In [101]:
from datasets import concatenate_datasets

paralel_400k = concatenate_datasets([paralel_377k, paralel_44k])

In [102]:
paralel_400k.save_to_disk("dataset/paralel_3_lang/paralel_dataset_400k")

Saving the dataset (4/4 shards): 100%|██████████| 392513/392513 [00:27<00:00, 14353.79 examples/s]


In [116]:
paralel_400k[456]

{'id': '826b756d-1984-4a9b-9e44-603ef375821c',
 'cirebonese': 'Benteng Tiworo punika salah satunggal badan peninggalan sajarah saking Karaton Tiworo. Badan lan abha saking kubu punika taksih bisa tag saksikan hinga babak yasa. Benteng Tiworo kapanggihan ing Kelurahan Waumere, anatap Tiworo Kepulauan, kabupaten Muna Barat, Sulawesi Tenggara.\n\nBenteng Tiworo kadamel saking watu-watu alit lan ageng ingkang asusun rapi, lan ngadeg kuwat ing pusér Kota anatap Tiworo Kepulauan. Dhuwuré bervariasi, antawis telu ngantos papat meter. Kubu punika akělit amba kirang langkung loro hektar. Lokasi punika pinakangga saking imamat Karaton Tiworo. Hinga sapunika lokasi Benteng Tiworopun taksih disaklarkan, sanadyan badan aslinipun sampun ngalami owah-owahan.\n\nSajarah\nMiturut sajarah, Benteng Tiworo dibangun ing abad XVI dening Raja Muna ingkang La Ode Asmana. Pembuatan kubu punika saking bahan watu ingkang dikangkat masyarakat kalawan cara serendeng sepanjang 150 kilometer. Watu-watu ingkang dipun

In [52]:
id_cbn = load_from_disk("dataset/paralel_2_lang/paralel_id_cbn_127k_dedup")

In [53]:
existing_id_text = set(paralel_377k["indonesian"])

In [54]:
def filter_id_cbn(data):
    return data["text"] not in existing_id_text

In [55]:
filtered_id_cbn = id_cbn.filter(filter_id_cbn)

Filter: 100%|██████████| 92504/92504 [00:13<00:00, 6913.87 examples/s]


In [57]:
filtered_id_cbn.save_to_disk("dataset/paralel_2_lang/paralel_id_cbn_127k_filtered")

Saving the dataset (1/1 shards): 100%|██████████| 44743/44743 [00:00<00:00, 53675.53 examples/s]


In [78]:
from datasets import load_dataset

id_mmlu = load_dataset("indolem/IndoMMLU")

In [79]:
bali_mmlu = id_mmlu["test"].filter(lambda x: x["subject"] == "Balinese")

In [80]:
bali_mmlu[0]

{'subject': 'Balinese',
 'group': 'Local languages and cultures',
 'level': 'SD',
 'class': '2',
 'question': 'Dugas Galungane titiang mabakti nganggon udeng baru. Udeng basa Bali alusne...',
 'options': "['A. destar', 'B. kampuh', 'C. wastra']",
 'answer': 'A',
 'is_for_fewshot': '0'}

In [81]:
import ast
import uuid

def transform_data(item):
    question_id = str(uuid.uuid4())

    # Parse the options string into a list
    options_str = item['options']
    options_list = ast.literal_eval(options_str)
    
    # Extract labels and texts from options
    labels = []
    texts = []
    for opt in options_list:
        # Split by dot and strip whitespace
        parts = opt.split('.', 1)
        labels.append(parts[0].strip().lower())  # Get 'A', 'B', 'C' and convert to lowercase
        texts.append(parts[1].strip())  # Get the actual option text
    
    # Create the transformed dictionary
    transformed = {
        "context": "",
        "question": item['question'],
        "choices": {
            "label": labels,
            "text": texts
        },
        "answer": item['answer'].lower(),  # Convert answer to lowercase to match labels
        "category": [],
        "grade": int(item['class']),
        "question_id": question_id
    }
    
    return transformed

In [82]:
bali_mmlu = bali_mmlu.map(transform_data, remove_columns=['options', 'class', 'subject', 'group', 'level', 'class', 'is_for_fewshot'])

Map: 100%|██████████| 471/471 [00:00<00:00, 7187.59 examples/s]


In [83]:
bali_mmlu[0]

{'question': 'Dugas Galungane titiang mabakti nganggon udeng baru. Udeng basa Bali alusne...',
 'answer': 'a',
 'context': '',
 'choices': {'label': ['a', 'b', 'c'], 'text': ['destar', 'kampuh', 'wastra']},
 'category': [],
 'grade': 2,
 'question_id': 'e22caaf5-7596-4d76-a04e-61637f60a8c1'}

In [84]:
bali_mmlu.save_to_disk("dataset/bali_mmlu")

Saving the dataset (1/1 shards): 100%|██████████| 471/471 [00:00<00:00, 60177.81 examples/s]


In [85]:
paralel_377k = load_from_disk("dataset/paralel_3_lang/paralel_dataset_377k_dedup")

In [86]:
paralel_377k

Dataset({
    features: ['id', 'cirebonese', 'indonesian', 'balinese'],
    num_rows: 348695
})

In [2]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o-mini")

def num_tokens_from_string(string: str, encoder) -> int:
    """Returns the number of tokens in a text string."""
    num_tokens = len(encoder.encode(string))
    return num_tokens

def count_tokens_in_dataset(dataset, field_name, num_tokens_from_string, encoder):
    total_tokens = 0
    
    for item in dataset:
        text = item[field_name]
        tokens = num_tokens_from_string(text, encoder)
        total_tokens += tokens
    
    return total_tokens

In [128]:
paralel_400k = load_from_disk("dataset/paralel_3_lang/paralel_dataset_400k_dedup")

In [129]:
print(count_tokens_in_dataset(paralel_400k, "indonesian", num_tokens_from_string, encoding))

168216484


In [130]:
print(count_tokens_in_dataset(paralel_400k, "cirebonese", num_tokens_from_string, encoding))

182488277


In [131]:
print(count_tokens_in_dataset(paralel_400k, "balinese", num_tokens_from_string, encoding))

184671868


In [249]:
paralel_400k = load_from_disk("dataset/id_hq/id_hq_data_prompt_300k")

In [266]:
import random

random.choice(paralel_400k)

{'text': 'Joel Robles Blázquez (lahir 17 Juni 1990), sing dikenal kanthi jeneng Joel, ya iku pemain bal-balan profésional Spanyol sing main dadi kiper kanggo klub Liga Utama Inggris Everton F.C. Robles miwiti karir profésional ing Atlético Madrid, wiwitané ing tim C ing taun 2008, sadurungé maju ing tim B ing taun sabanjuré, banjur diunggahaké dadi pemain senior. Dhèwèké dipindhah menyang Rayo Vallecano ing Januari 2012, lan setaun sangarepé nggabung karo klub Liga Utama Wigan Athletic, ing endi dhèwèké dadi bagéyan saka pamenang FA Cup Final 2013 sadurungé gabung karo Everton.\n\nRobles main ing pertandhingan internasional kanggo Spanyol ing tingkat U-16, U-17, U-21 lan U-23, lan dadi bagéan saka skuad sing menang ing Kejuaraan Eropah U-21 2013.\n\nKarir klub\n\nAtlético Madrid\nLair ing Getafe, Madrid, Robles kalebu asil produk pelatihan remaja ing Kutha Getafe. Dhèwèké gabung karo Atlético Madrid ing taun 2005. Dhèwèké ngrampungaké mangsa profésional kapisané ing mangsa (2009-10), d

In [248]:
random.choice(paralel_300k)

{'id': 'a1c3096e-943c-4787-9870-18017e6a701b',
 'cirebonese': 'Ari 2002 XG4 mangrupa hiji astéroid. Ieu asteroid téh bagéan tina astéroid Amor, anu nganjrek deukeut jeung marcapada. Ékséntrisitas orbit ieu astéroid kacatet gedéna 0.480, sedengkeun magnitudo mutlakna 18.2. Ari nu cios référénsina mah nyaéta MPO 237836.\n\nBebentukan\nKawas sakumna astéroid, ieu astéroid kabentuk tina nébula panonpoé primordial minangka beubeulahan planétisimal, objék di nébula marcapada ngora nu teu cukup badag pikeun robah jadi planét.\n\nRujukan\n\nTutumbu kaluar\nDaptar astéroid Amor - The sejagat Astronomical Union Minor Planet Center.\n\n338347\n338347',
 'indonesian': 'Ari 2002 XG4 mangrupa hiji astéroid. Ieu asteroid téh bagéan tina astéroid Amor, anu nganjrek deukeut jeung marcapada. Ékséntrisitas orbit ieu astéroid kacatet gedéna 0.480, sedengkeun magnitudo mutlakna 18.2. Ari nu jadi référénsina mah nyaéta MPO\xa0237836.\n\nBebentukan\nKawas sakumna astéroid, ieu astéroid kabentuk tina nébula p

In [3]:
import fasttext

# load the model
model = fasttext.load_model("./models/glotlid/model.bin")

In [4]:
import fasttext
import numpy as np
class CustomLID:
    def __init__(self, model_path, languages = -1, mode='before'):
        self.model = fasttext.load_model(model_path)
        self.output_matrix = self.model.get_output_matrix()
        self.labels = self.model.get_labels()
        
        # compute language_indices
        if languages !=-1 and isinstance(languages, list):
            self.language_indices = [self.labels.index(l) for l in list(set(languages)) if l in self.labels]

        else:
            self.language_indices = list(range(len(self.labels)))

        # limit labels to language_indices
        self.labels = list(np.array(self.labels)[self.language_indices])
        
        # predict
        self.predict = self.predict_limit_after_softmax if mode=='after' else self.predict_limit_before_softmax

    
    def predict_limit_before_softmax(self, text, k=1):
        
        # sentence vector
        sentence_vector = self.model.get_sentence_vector(text)
        
        # dot
        result_vector = np.dot(self.output_matrix[self.language_indices, :], sentence_vector)

        # softmax
        softmax_result = np.exp(result_vector - np.max(result_vector)) / np.sum(np.exp(result_vector - np.max(result_vector)))

        # top k predictions
        top_k_indices = np.argsort(softmax_result)[-k:][::-1]
        top_k_labels = [self.labels[i] for i in top_k_indices]
        top_k_probs = softmax_result[top_k_indices]

        return tuple(top_k_labels), top_k_probs


    def predict_limit_after_softmax(self, text, k=1):
        
        # sentence vector
        sentence_vector = self.model.get_sentence_vector(text)
        
        # dot
        result_vector = np.dot(self.output_matrix, sentence_vector)

        # softmax
        softmax_result = np.exp(result_vector - np.max(result_vector)) / np.sum(np.exp(result_vector - np.max(result_vector)))

        # limit softmax to language_indices
        softmax_result = softmax_result[self.language_indices]

        
        # top k predictions
        top_k_indices = np.argsort(softmax_result)[-k:][::-1]
        top_k_labels = [self.labels[i] for i in top_k_indices]
        top_k_probs = softmax_result[top_k_indices]

        return tuple(top_k_labels), top_k_probs


In [5]:
# to make sure these languages are available in GlotLID check the list of supported labels in model.labels
limited_languages = ['__label__ind_Latn', '__label__sun_Latn', '__label__jav_Latn', '__label__ban_Latn']

model = CustomLID("./models/glotlid/model.bin", languages = limited_languages , mode='before')

In [75]:
pred = model.predict("halo halo bandung ibu kota priangan", 3)

In [246]:
import random

model.predict(random.choice(paralel_705k)["balinese"].split("\n")[-1])

(('__label__ind_Latn',), array([0.99998975], dtype=float32))

In [55]:
import datasets

# 1. Load the dataset
paralel_705k = datasets.load_from_disk("dataset/paralel_3_lang/combined_paralel_dataset_705k_dedup")

In [247]:
def filter_non_id_text(data):
    sentences = data["indonesian"].split("\n")
    first_line = sentences[0]
    first_pred = model.predict(first_line)[0][0]
    if len(sentences) == 1:
      if first_pred == "__label__ind_Latn":
          return None
      else:
          return data
    else:
      second_line = sentences[1]
      second_pred = model.predict(second_line)[0][0]
      if first_pred == "__label__ind_Latn" and second_pred == "__label__ind_Latn":
          return None
      else:
          return data

def filter_id_text(data):
    sentences = data["indonesian"].split("\n")
    first_line = sentences[0]
    first_pred = model.predict(first_line)[0][0]
    if len(sentences) == 1:
      if first_pred == "__label__ind_Latn":
          return data
      else:
          return None
    else:
      second_line = sentences[1]
      second_pred = model.predict(second_line)[0][0]
      if first_pred == "__label__ind_Latn" and second_pred == "__label__ind_Latn":
          return data
      else:
          return None

def filter_id_text_2(data):
    sentences = data["text"].split("\n")
    first_line = sentences[0]
    first_pred = model.predict(first_line)[0][0]
    if len(sentences) == 1:
      if first_pred == "__label__ind_Latn":
          return data
      else:
          return None
    else:
      second_line = sentences[1]
      second_pred = model.predict(second_line)[0][0]
      if first_pred == "__label__ind_Latn" and second_pred == "__label__ind_Latn":
          return data
      else:
          return None

def filter_all_text(data):
    cbn_sentences = data["cirebonese"].split("\n")
    ban_sentences = data["balinese"].split("\n")
    
    cbn_first_line = cbn_sentences[0]
    cbn_first_pred = model.predict(cbn_first_line)[0][0]
    cbn_verdict = False
    if len(cbn_sentences) == 1:
      if cbn_first_pred == "__label__jav_Latn":
          cbn_verdict = True
    else:
      cbn_second_line = cbn_sentences[1]
      cbn_second_pred = model.predict(cbn_second_line)[0][0]
      if cbn_first_pred == "__label__jav_Latn" or cbn_second_pred == "__label__jav_Latn":
          cbn_verdict = True

    ban_first_line = ban_sentences[0]
    ban_first_pred = model.predict(ban_first_line)[0][0]
    ban_verdict = False
    if len(ban_sentences) == 1:
      if ban_first_pred == "__label__ban_Latn":
          ban_verdict = True
    else:
      ban_second_line = ban_sentences[1]
      ban_second_pred = model.predict(ban_second_line)[0][0]
      if ban_first_pred == "__label__ban_Latn" or ban_second_pred == "__label__ban_Latn":
          ban_verdict = True

    if cbn_verdict and ban_verdict:
        return data
    else:
        return None

In [248]:
clean_paralel_705k = paralel_705k.filter(filter_all_text, num_proc=8)

Filter (num_proc=8): 100%|██████████| 557859/557859 [01:21<00:00, 6808.73 examples/s]


In [250]:
clean_paralel_705k

Dataset({
    features: ['id', 'cirebonese', 'indonesian', 'balinese'],
    num_rows: 491113
})

In [2]:
clean_paralel_705k = load_from_disk("dataset/paralel_3_lang/combined_paralel_dataset_705k_dedup_clean")

In [392]:
def filter_by_word_length(example):
    word_count_cbn = len(example['cirebonese'].split())
    word_count_bali = len(example['balinese'].split())
    return 20 <= word_count_cbn <= 50 and 20 <= word_count_bali <= 50

In [4]:
# Filter the dataset
filtered_dataset = clean_paralel_705k.filter(filter_by_word_length)

Filter: 100%|██████████| 491113/491113 [01:05<00:00, 7551.89 examples/s] 


In [68]:
import random

random.choice(filtered_dataset)

{'id': '60304c80-13e3-4a79-88bf-722ba2650aaf',
 'cirebonese': 'HAT-P-1b punika sěkěrět planet jaba surya ingkang terletak sukat 520,92 taun dilah saking Bumi. Planet niki akělit pada taun 2006 kalawan ngginakaken metode transit. HAT-P-1b akělit massa amun 0,525 massa Jupiter.\n\nReferensi \n\nPlanet jaba surya',
 'indonesian': 'HAT-P-1b adalah sebuah planet luar surya yang terletak sekitar 520,92 tahun cahaya dari Bumi. Planet ini ditemukan pada tahun 2006 dengan menggunakan metode  transit. HAT-P-1b memiliki massa sebesar 0,525 massa Jupiter.\n\nReferensi \n \n\nPlanet luar surya',
 'balinese': 'HAT-P-1b punika abulih planet jaba surya sane terletak sawatara 520,92 tahun dipta saking Bumi. Planet niki kangen ri tahun 2006 ajak makebah metode transit. HAT-P-1b madue massa amun 0,525 massa Jupiter.\n\nReferensi \n\nPlanet jaba surya'}

## Create Annotation Data

In [316]:
bali_paralel = load_from_disk("dataset/paralel_3_lang/paralel_dataset_22k")

In [393]:
bali_med = bali_paralel.filter(filter_by_word_length)

Filter: 100%|██████████| 22248/22248 [00:01<00:00, 17114.46 examples/s]


In [445]:
random.choice(bali_med)

{'id': '816a70ef-a61a-4b83-ad48-ee4f50b77804',
 'balinese': 'Ketut Santos Fernandez uli cerik jenek di Lampung. Uling cerik biasa tuturina ane serem-serem teken reramane. Gumi Bali ento tenget, kerana sabilang wai krama Baline jemet mabanten.',
 'indonesian': 'Ketut Santos Fernandez sejak kanak-kanak tinggal di Lampung. Semenjak usia dini selalu diceritakan hal yang mistis oleh orangtuanya. Pulau Bali disebut angker sebab setiap hari warga Bali rajin sembahyang dan menghaturkan banten.',
 'prompt': 'Translate the given Indonesian text in the <id_text> tag below into Cirebonese with the help of some word-to-word translation provided below. For one word, there can be multiple translations, and you need to choose the right one based on the context. The translations are as follows:\n- setiap hari: sabên, saban dina, amban dina, unggal dina, saban\n- sejak: atêwêk\n- tinggal: tilar, tinggal\n- di: teng, ning, dipun, di, ada\n- semenjak: jég, sajég\n- usia: umur, ayusa, ayusya\n- selalu: kad

In [319]:
filter_short = load_from_disk("dataset/paralel_3_lang/filtered_paralel_dataset_90k_short")

In [494]:
banned_words = ["referensi", "pranala luar", "lihat juga", "lihat pula", "rujukan", "catatan kaki", "pranala", "kota", "kecamatan", "kelurahan", "kabupaten", "negara", "provinsi", "desa", "adalah", "merujuk"]

def filter_id_text(data):
    for word in banned_words:
        if word in data["indonesian"].lower():
            return None
    return data

In [495]:
clean_short = filter_short.filter(filter_id_text)

Filter: 100%|██████████| 93748/93748 [00:04<00:00, 20814.66 examples/s]


In [497]:
clean_short

Dataset({
    features: ['id', 'cirebonese', 'indonesian', 'balinese'],
    num_rows: 1714
})

In [513]:
random.choice(clean_short)

{'id': 'e4e2ec19-1ae4-4eae-bd49-490dd01368fc',
 'cirebonese': 'Jujuluk Natakusuma, sěkěrět jujuluk kepangeranan ingkang gage atêwêk adêg Amangkurat II ning Kesultanan Mataram, Jawa. Natakusuma, nama asli Paku Alam I.',
 'indonesian': 'Gelar Natakusuma, sebuah gelar kepangeranan yang dimulai sejak pemerintahan Amangkurat II di Kesultanan Mataram, Jawa.\n Natakusuma, nama asli Paku Alam I.',
 'balinese': 'Desak Natakusuma, abulih desak kepangeranan sane ngametuang sasukate pemerintahan Amangkurat II ring Kesultanan Mataram, Jawa. Natakusuma, aran asli Paku Alam I.'}

In [515]:
clean_short.save_to_disk("dataset/paralel_3_lang/filtered_paralel_dataset_1k_short_nowiki")

Saving the dataset (1/1 shards): 100%|██████████| 1714/1714 [00:00<00:00, 108252.33 examples/s]


In [516]:
import json

def load_dictionary(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

In [517]:
bali_indo_dict = load_dictionary("dict/bali_idn.json")
indo_bali_dict = load_dictionary("dict/idn_bali.json")
cbn_indo_dict = load_dictionary("dict/cbn_idn.json")
indo_cbn_dict = load_dictionary("dict/idn_cbn.json")

In [518]:
import re
import random

def extract_symbols(word):
    # Extract prefix and suffix symbols
    prefix = re.match(r'^[^\w\'`êÊ]*', word).group(0)
    suffix = re.search(r'[^\w\'`êÊ]*$', word).group(0)
    return prefix, suffix

def clean_word(word):
    # Keep alphanumeric, apostrophes, backticks and ê
    cleaned = re.sub(r'[^\w\'`êÊ]', '', word)
    return cleaned

def correct_and_analyze_text(text, to_idn_dict, from_idn_dict):
    words = text.split()
    corrected_words = []
    valid_count = 0
    corrected_count = 0
    correction_dict = {}

    for word in words:
        original_word = word
        prefix, suffix = extract_symbols(word)
        word = clean_word(word)
        
        if word.islower():
            if word.lower() in to_idn_dict:
                corrected_words.append(original_word)
                valid_count += 1
            else:
                translations = from_idn_dict.get(word.lower(), None)
                if translations:
                    replacement = translations[0] if translations else random.choice(translations)
                    # Add back the symbols to the translation
                    replacement = prefix + replacement + suffix
                    corrected_words.append(replacement)
                    corrected_count += 1
                    correction_dict[original_word] = replacement
                else:
                    corrected_words.append(original_word)
        else:
            corrected_words.append(original_word)
            valid_count += 1
    
    total_words = len(words)
    invalid_count = total_words - valid_count - corrected_count
    
    return {
        'corrected_text': ' '.join(corrected_words),
        'valid_percentage': (valid_count / total_words) * 100,
        'invalid_percentage': (invalid_count / total_words) * 100,
        'corrected_percentage': (corrected_count / total_words) * 100,
        'corrections': correction_dict
    }

In [519]:
def correct_and_analyze_text_ban_dataset(data):
    result = correct_and_analyze_text(
        data['balinese'],
        bali_indo_dict,
        indo_bali_dict
    )
    
    return {
        'balinese': data['balinese'].strip(),
        'indonesian': data['indonesian'].strip(),
        'balinese_corrected': result['corrected_text'],
        'valid_percentage': result['valid_percentage'],
        'invalid_percentage': result['invalid_percentage'],
        'corrected_percentage': result['corrected_percentage'],
        'corrections': list(result['corrections'].items()),
    }

def correct_and_analyze_text_cbn_dataset(data):
    result = correct_and_analyze_text(
        data['cirebonese'],
        cbn_indo_dict,
        indo_cbn_dict
    )
    
    return {
        'cirebonese': data['cirebonese'].strip(),
        'indonesian': data['indonesian'].strip(),
        'cirebonese_corrected': result['corrected_text'],
        'valid_percentage': result['valid_percentage'],
        'invalid_percentage': result['invalid_percentage'],
        'corrected_percentage': result['corrected_percentage'],
        'corrections': list(result['corrections'].items()),
    }

In [520]:
sample = clean_short.shuffle(seed=42).select(range(600))

In [521]:
sample_1 = sample.select(range(300))
sample_2 = sample.select(range(300, 600))

In [522]:
ban_sample = sample_2.map(correct_and_analyze_text_ban_dataset, remove_columns=['cirebonese'])
cbn_sample = sample_1.map(correct_and_analyze_text_cbn_dataset, remove_columns=['balinese'])

Map: 100%|██████████| 300/300 [00:00<00:00, 6936.02 examples/s]


In [571]:
random.choice(ban_sample)

{'id': 'f90d59f1-cfe1-4297-a65f-0c6f2c2d7256',
 'indonesian': 'Pintu air (pelayaran), perangkat untuk mengatur kedalaman pada alur pelayaran.\n Pintu air (floodgate), perangkat untuk mengontrol aliran air di sungai, kanal, atau waduk.\n Sluis, perangkat untuk mengontrol tinggi aliran air di sungai atau kanal.',
 'balinese': 'Apes yeh (pelayaran), prajuru buat ngatur kedalaman ri celocoh pelayaran. Apes yeh (gembok banjir), prajuru buat ngontrol kecoran yeh ring tukad, kanal, utawi waduk. Sluis, prajuru buat ngontrol ganggas kecoran yeh ring tukad utawi kanal.',
 'balinese_corrected': 'Apes yeh (pelayaran), prajuru buat ngatur kedalaman ri celocoh pelayaran. Apes yeh (gembok blabar), prajuru buat ngontrol kecoran yeh ring tukad, kanal, utawi waduk. Sluis, prajuru buat ngontrol ganggas kecoran yeh ring tukad utawi kanal.',
 'valid_percentage': 77.14285714285715,
 'invalid_percentage': 20.0,
 'corrected_percentage': 2.857142857142857,
 'corrections': [['banjir),', 'blabar),']]}

In [572]:
random.choice(cbn_sample)

{'id': '6dcb330f-fde0-4513-a748-6956c63757e9',
 'cirebonese': 'Orde Anyar bisa ngacu ing pirang-pirang bab ing ngandhap: \n\n Orde Anyar, têngêr dum mangsa imamat Presiden Nusantara ke-2 Soeharto. \n Orde Anyar, order politik ning Jerman Nazi.',
 'indonesian': 'Orde Baru dapat mengacu pada beberapa hal berikut:\n\n Orde Baru, sebutan bagi masa pemerintahan Presiden Indonesia ke-2 Soeharto.\n Orde Baru, tatanan politik di Jerman Nazi.',
 'cirebonese_corrected': 'Orde Anyar bisa ngacu ing pirang-pirang bab ing ngandhap: Orde Anyar, têngêr dum mangsa imamat Presiden Nusantara ke-2 Soeharto. Orde Anyar, order politik ning Jerman Nazi.',
 'valid_percentage': 88.46153846153845,
 'invalid_percentage': 11.538461538461538,
 'corrected_percentage': 0.0,
 'corrections': []}

In [573]:
def dataset_to_csv(dataset, output_file):
    # Convert corrections list of tuples to dictionary string format
    def format_corrections(corrections):
        return '\n'.join(f"{orig} -> {corr}" for orig, corr in corrections)
    
    # Convert to pandas DataFrame
    df = dataset.to_pandas()
    
    # Format the corrections column
    if 'corrections' in df.columns:
        df['corrections'] = df['corrections'].apply(format_corrections)
    
    # Save to CSV
    df.to_csv(output_file, index=False, encoding='utf-8')
    
    print(f"Dataset saved to {output_file}")
    print(f"Total rows: {len(df)}")

In [574]:
dataset_to_csv(ban_sample, "dataset/annotation/ban_sample_300/ban.csv")
dataset_to_csv(cbn_sample, "dataset/annotation/cbn_sample_300/cbn.csv")

Dataset saved to dataset/annotation/ban_sample_300/ban.csv
Total rows: 300
Dataset saved to dataset/annotation/cbn_sample_300/cbn.csv
Total rows: 300


In [575]:
ban_sample.save_to_disk("dataset/annotation/ban_sample_300")
cbn_sample.save_to_disk("dataset/annotation/cbn_sample_300")

Saving the dataset (1/1 shards): 100%|██████████| 300/300 [00:00<00:00, 74441.89 examples/s]


## Gathering Previous High-Quality Balinese Data

In [4]:
from huggingface_hub import login
login()

In [5]:
import datasets
from datasets import load_dataset

dset = datasets.load_dataset("cis-lmu/udhr-lid", trust_remote_code=True)

README.md:   0%|          | 0.00/6.45k [00:00<?, ?B/s]

udhr-lid.csv:   0%|          | 0.00/7.07M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/27757 [00:00<?, ? examples/s]

In [10]:
bali_udhr = dset['test'].filter(lambda x: x['iso639-3'] == 'ban')

Filter:   0%|          | 0/27757 [00:00<?, ? examples/s]

In [14]:
bali_udhr.save_to_disk("dataset/bali_hq/bali_udhr")

Saving the dataset (0/1 shards):   0%|          | 0/60 [00:00<?, ? examples/s]

In [15]:
bali_sib200 = load_dataset("Davlan/sib200", "ban_Latn")

README.md:   0%|          | 0.00/47.9k [00:00<?, ?B/s]

train.tsv:   0%|          | 0.00/114k [00:00<?, ?B/s]

dev.tsv:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

test.tsv:   0%|          | 0.00/33.1k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [17]:
bali_sib200["train"]

Dataset({
    features: ['index_id', 'category', 'text'],
    num_rows: 701
})

In [18]:
bali_sib200["train"].save_to_disk("dataset/bali_hq/bali_sib200")

Saving the dataset (0/1 shards):   0%|          | 0/701 [00:00<?, ? examples/s]

In [19]:
bali_glot500 = dataset = load_dataset('cis-lmu/Glot500', 'ban_Latn', split='train')

README.md:   0%|          | 0.00/48.8k [00:00<?, ?B/s]

data-00000-of-00001.arrow:   0%|          | 0.00/8.10M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [21]:
bali_glot500.save_to_disk("dataset/bali_hq/bali_glot500") 

Saving the dataset (0/1 shards):   0%|          | 0/48958 [00:00<?, ? examples/s]

In [23]:
bali_madlad = load_dataset("allenai/madlad-400", "ban", trust_remote_code=True)

ban_clean_0000.jsonl.gz:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

ban_noisy_0000.jsonl.gz:   0%|          | 0.00/12.6M [00:00<?, ?B/s]

Generating clean split: 0 examples [00:00, ? examples/s]

Generating noisy split: 0 examples [00:00, ? examples/s]

In [26]:
bali_madlad["clean"].save_to_disk("dataset/bali_hq/bali_madlad")

Saving the dataset (0/1 shards):   0%|          | 0/637 [00:00<?, ? examples/s]

In [30]:
bali_nllb = load_dataset("acul3/KoPI-NLLB", "ban_Latn-neardup", trust_remote_code=True)

Repo card metadata block was not found. Setting CardData to empty.


ban_Latn.json.zst:   0%|          | 0.00/23.0M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [31]:
bali_nllb["train"].save_to_disk("dataset/bali_hq/bali_nllb")

Saving the dataset (0/1 shards):   0%|          | 0/244545 [00:00<?, ? examples/s]

In [40]:
import pandas as pd
from datasets import Dataset

# Read the CSV file
df = pd.read_csv("dataset/bali_hq/id-ban.tsv", index_col=0, sep="\t")

df = df.dropna()
df['Balinese'] = df['Balinese'].astype(str)
# Extract only the balinese column and convert to dict format
balinese_data = {
    'text': df['Balinese'].tolist()
}

# Convert to HuggingFace dataset
balinese_dataset = Dataset.from_dict(balinese_data)

# Save to disk
balinese_dataset.save_to_disk("dataset/bali_hq/bali_indonmt")

Saving the dataset (0/1 shards):   0%|          | 0/5165 [00:00<?, ? examples/s]

In [36]:
balinese_dataset

Dataset({
    features: ['text'],
    num_rows: 20611
})

In [42]:
from datasets import load_from_disk

bali_indonmt = load_from_disk("dataset/bali_hq/bali_indonmt")
bali_nusax = load_from_disk("dataset/bali_hq/bali_nusax")
bali_wiki = load_from_disk("dataset/bali_hq/bali_wiki")

In [62]:
def convert_dataset(data):
    data["text"] = data["sentence"]
    return data

In [63]:
bali_udhr = bali_udhr.map(convert_dataset, remove_columns=["sentence", "id", "iso639-3", "iso15924", "language"])

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

In [64]:
bali_udhr

Dataset({
    features: ['text'],
    num_rows: 60
})

In [65]:
bali_wiki

Dataset({
    features: ['text'],
    num_rows: 20611
})

In [67]:
from datasets import concatenate_datasets

all_bali_hq = concatenate_datasets([bali_udhr, bali_sib200, bali_glot500, bali_madlad, bali_nllb, bali_indonmt, bali_nusax, bali_wiki])

In [70]:
all_bali_hq = all_bali_hq.remove_columns(['index_id', 'category', 'dataset', 'script', 'lang_script', 'url', 'score', 'source'])

In [72]:
all_bali_hq.save_to_disk("dataset/bali_hq/all_bali_hq")

Saving the dataset (0/1 shards):   0%|          | 0/280246 [00:00<?, ? examples/s]

In [6]:
def filter_bali_text(data):
    ban_sentences = data["text"].split("\n")
    
    ban_first_line = ban_sentences[0]
    ban_first_pred = model.predict(ban_first_line)[0][0]
    ban_verdict = False
    if len(ban_sentences) == 1:
      if ban_first_pred == "__label__ban_Latn":
          ban_verdict = True
    else:
      ban_second_line = ban_sentences[1]
      ban_second_pred = model.predict(ban_second_line)[0][0]
      if ban_first_pred == "__label__ban_Latn" or ban_second_pred == "__label__ban_Latn":
          ban_verdict = True

    if ban_verdict:
        return data
    else:
        return None

In [7]:
all_bali_hq_dedup = load_from_disk("dataset/bali_hq/all_bali_hq_dedup")

In [8]:
all_bali_hq_clean = all_bali_hq_dedup.filter(filter_bali_text, num_proc=8)

Parameter 'function'=<function filter_bali_text at 0x7f632236fb50> of the transform datasets.arrow_dataset.Dataset.filter@2.0.1 couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Filter (num_proc=8): 100%|██████████| 262601/262601 [00:11<00:00, 22536.63 examples/s]


In [10]:
count_tokens_in_dataset(all_bali_hq_clean, "text", num_tokens_from_string, encoding)

11075340

In [11]:
all_bali_hq_clean.save_to_disk("dataset/bali_hq/all_bali_hq_clean")

Saving the dataset (1/1 shards): 100%|██████████| 201404/201404 [00:00<00:00, 207678.31 examples/s]


In [14]:
from datasets import load_dataset

# or for a specific language
bali_ift = load_dataset("akoksal/muri-it-language-split", "ban")

Generating test split: 100%|██████████| 335/335 [00:00<00:00, 77974.02 examples/s]


In [17]:
bali_ift =  bali_ift["train"]
bali_ift.save_to_disk("dataset/bali_ift/bali_muri")

Saving the dataset (1/1 shards): 100%|██████████| 6026/6026 [00:00<00:00, 603449.43 examples/s]


## English High-Quality Data

In [12]:
en_wiki = load_from_disk("dataset/en_hq/en_wiki")

In [13]:
count_tokens_in_dataset(en_wiki, "text", num_tokens_from_string, encoding)

5615855